##  Rozwiązywanie układu równań liniowych metodami rozkładu LU 

Podaj zasadę działania metod opartych o dekompozycję LU. 

#### Zadanie 1
Zaimplementuj metody rozkładu LU:
- Rozkład Crouta 
- Rozkład Doolitla
- Rozkład Choleskyego 

Dla każdej z metod podaj warunki niezbędne aby można ją było zastosować. Sprawdź poprawność działania tych metod. 

Przetestuj wydajność algorytmów dla kilku rozmiarów macierzy (podobnie jak w ćwiczeniu 9). 




Dekompozycja LU - jest to jedna z metod rozwiązywania układu równań nieliniowych, nazwa ta pochodzi od użytych w tej metodzie macierzy trójkątnych tj. dolnotrójkątnej(L) oraz górnotrójkątnej(U).   
Niech dany będzie układ równań liniowych:    
$A * x = y$      
,gdzie $A$ – macierz współczynników, $x$ – wektor niewiadomych, $y$ – wektor danych(wyrazów wolnych).

W metodzie LU macierz współczynników $A$ zapisywana jest jako iloczyn pewnych macierzy dolnej $L$ i górnej $U$

$A = L*U$   
, gdzie :
![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/0365e58fcda12c0e33a5509d37fd0658cefba13d) 

Układ równań przyjmuje wówczas postać :

$L*U*x = y$         
a jego rozwiązanie sprowadza się do rozwiązania dwóch układów równań z macierzami trójkątnymi:

$L * z = y$   
$U * x = z$

- **Rozkład Crouta** - aby zastosować rozkład Crouta musi zostać spełniony następujący warunek, na diagonalnej macierzy trójkątnej dolnej $U$ muszą występować wyłacznie 1 ($u_{ii} = 1$).


In [0]:
import math
import time
import numpy as np


def crout(A):
    n = len(A)
    L = [[0] * n for i in range(n)]
    U = [[0] * n for i in range(n)]

    for j in range(n):
        U[j][j] = 1
        for i in range(j, n):
            alpha = float(A[i][j])
            for k in range(j):
                alpha -= L[i][k] * U[k][j]
            L[i][j] = alpha
        for i in range(j + 1, n):
            tmpU = float(A[j][i])
            for k in range(j):
                tmpU -= L[j][k] * U[k][i]
            if int(L[j][j]) == 0:
                L[j][j] = 0
            U[j][i] = tmpU / L[j][j]
    return [L, U]

- **Rozkład Doolitla** - aby zastosować rozkład Dollitla musi zostać spełniony następujący warunek, na diagonalnej macierzy trójkątnej górnej $L$ muszą występować wyłacznie 1 ($l_{ii} = 1$).

In [0]:
def dollitle(A):
    n = len(A)
    L = [[0] * n for i in range(n)]
    U = [[0] * n for i in range(n)]

    for j in range(n):
        L[j][j] = 1.0
        for i in range(j + 1):
            s1 = sum(U[k][j] * L[i][k] for k in range(i))
            U[i][j] = A[i][j] - s1
        for i in range(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in range(j))
            L[i][j] = (A[i][j] - s2) / U[j][j]

    return [L, U]

- **Rozkład Choleskyego** - aby zastosować rozkład Choleskyego muszą zostać spełniony następujący waruneki,   
 - wartości na diagonalnej macierzy trójkątnej górnej $L$ oraz diagonalnej   macierz trójkątnej dolnej $U$  muszą być sobie równe ($u_{ii} = l_{ii}$),
 - macierz $A$ musi być symetryczna oraz dodatnie określona.

In [0]:
def cholesky(A):
    n = len(A)
    L = [[0.0] * n for i in range(n)]

    for i in range(n):
        for k in range(i + 1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            if (i == k):
                L[i][k] = math.sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return [L, np.array(L).transpose()]

Poprawności działania tych metod zbadam na macierzy 3x3 oraz stworzyłem dodatkową metode create_rand_symmetric_and_positive_matrix, aby zapewnić spełnienie warunków dla metody cholesky 

In [0]:
def solve_equtition(A, B, fun):
    [L, U] = fun(A)
    z = np.linalg.solve(L, B)
    x = np.linalg.solve(U, z)
    return x


def create_rand_matrix(min, max, cols, rows):
    return np.random.randint(low=min, high=max + 1, size=(rows, cols))


def create_rand_vector(min, max, size):
    return np.random.randint(low=min, high=max + 1, size=(size,))


def create_rand_symmetric_and_positive_matrix(min, max, cols, rows):
    A = create_rand_matrix(min, max, cols, rows)
    B = np.dot(A, A.transpose())
    return (B + B.T) / 2


def print_matrix(A):
    for i in A:
        print(i)

In [11]:
A = create_rand_symmetric_and_positive_matrix(3, 10, 3, 3)
Y = create_rand_vector(3, 10, 3)
[L_crout, U_crout] = crout(A)
[L_doll, U_doll] = dollitle(A)
[L_chol, U_chol] = cholesky(A)

print("Macierz A ma postać :")
print_matrix(A)
print("Macierz L dla rozkładu Crouta:")
print_matrix(L_crout)
print("Macierz U dla rozkładu Crouta:")
print_matrix(U_crout)
print("Macierz L dla rozkładu Dollitla:")
print_matrix(L_doll)
print("Macierz U dla rozkładu Dollitla:")
print_matrix(U_doll)
print("Macierz L dla rozkładu Choleskyego:")
print_matrix(L_chol)
print("Macierz U dla rozkładu Choleskyego:")
print_matrix(U_chol)

Macierz A ma postać :
[ 74. 123.  48.]
[123. 245.  99.]
[48. 99. 43.]
Macierz L dla rozkładu Crouta:
[74.0, 0, 0]
[123.0, 40.55405405405406, 0]
[48.0, 19.216216216216225, 2.7594135288237247]
Macierz U dla rozkładu Crouta:
[1, 1.662162162162162, 0.6486486486486487]
[0, 1, 0.4738420526491167]
[0, 0, 1]
Macierz L dla rozkładu Dollitla:
[1.0, 0, 0]
[1.662162162162162, 1.0, 0]
[0.6486486486486487, 0.4738420526491167, 1.0]
Macierz U dla rozkładu Dollitla:
[74.0, 123.0, 48.0]
[0, 40.55405405405406, 19.216216216216225]
[0, 0, 2.7594135288237283]
Macierz L dla rozkładu Choleskyego:
[8.602325267042627, 0.0, 0.0]
[14.298459565489772, 6.368206502152238, 0.0]
[5.579886659703326, 3.01752404067327, 1.6611482561239743]
Macierz U dla rozkładu Choleskyego:
[ 8.60232527 14.29845957  5.57988666]
[0.         6.3682065  3.01752404]
[0.         0.         1.66114826]


Analizując powyższe wyniki widzimy, że każdy warunek dla danej metody został spełniony.         
Następnie przeanalizuje czy wszystkie powyższe metody dają poprawnę rozwiązanie : 

In [12]:
def solve_equtition(A, Y, fun):
    [L, U] = fun(A)
    z = np.linalg.solve(L, Y)
    x = np.linalg.solve(U, z)
    return x

print("Metoda Crouta")
print(solve_equtition(A, Y, crout))
print("Metoda Dollitla")
print(solve_equtition(A, Y, dollitle))
print("Metoda Choleskyego")
print(solve_equtition(A, Y, cholesky))

Metoda Crouta
[ 0.66731071 -1.17485811  2.16930322]
Metoda Dollitla
[ 0.66731071 -1.17485811  2.16930322]
Metoda Choleskyego
[ 0.66731071 -1.17485811  2.16930322]


Widzimy również, że wszystkie 3 metody dają identyczne poprawne rozwiązanie.

Do testowania wydajności powyższych funkcji zastowałem poniższą metodę : 

In [0]:
def compare_exec_time(min, max, N):
    A = create_rand_symmetric_and_positive_matrix(min, max, N, N)
    B = create_rand_vector(min, max, N)
    start_time = time.time()
    solve_equtition(A, B, crout)
    print("---LU Crouta took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    solve_equtition(A, B, dollitle)
    print("---LU Doolitla took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    solve_equtition(A, B, cholesky)
    print("---LU Choleskyego took %s seconds ---" % (time.time() - start_time))

Wyniki testów wykonałem poza środowiskiem jupytera, a same wyniki są nastepujące :     
- Dla macierzy 100x100 zakres losowanych liczb [0, 500] :    
  ---LU Crouta took 0.059658050537109375 seconds ---     
  ---LU Doolitla took 0.1942441463470459 seconds ---     
  ---LU Choleskyego took 0.14393281936645508 seconds ---    
- Dla macierzy 200x200 zakres losowanych liczb [0, 500] :    
  ---LU Crouta took 0.29308199882507324 seconds ---     
  ---LU Doolitla took 0.7315621376037598 seconds ---     
  ---LU Choleskyego took 0.40914058685302734 seconds ---   
- Dla macierzy 300x300 zakres losowanych liczb [0, 500] :     
  ---LU Crouta took 0.8489158153533936 seconds ---    
  ---LU Doolitla took 2.247528553009033 seconds ---     
  ---LU Choleskyego took 1.131791353225708 seconds ---  
- Dla macierzy 500x500 zakres losowanych liczb [0, 500] :      
  ---LU Crouta took 3.9642786979675293 seconds ---     
  ---LU Doolitla took 9.914392709732056 seconds ---      
  ---LU Choleskyego took 4.811501741409302 seconds ---     
- Dla macierzy 1000x1000 zakres losowanych liczb [0, 500] :     
  ---LU Crouta took 37.514655351638794 seconds ---    
  ---LU Doolitla took 88.99602842330933 seconds ---      
  ---LU Choleskyego took 38.43804574012756 seconds ---     

#### Zadanie 2 (dla chętnych)

#### Zadanie 3 
Zapoznaj się z funkcją rozwiązywania układów równań liniowych dostarczoną przez bibliotekę numpy/scipy. Porównaj jej wydajność z własnymi implementacjami.  



Biblioteki numpy oraz scipy dostarczają nam funkcję $linalg.solve()$ służącą do rozwiązywania równań.  
Do porównania wydajności tych funkcji z moją implementacją użyłem następującej funkncji :    

In [0]:
def compare_exec_time_2(min, max, N):
    A = create_rand_symmetric_and_positive_matrix(min, max, N, N)
    B = create_rand_vector(min, max, N)
    start_time = time.time()
    solve_equtition(A, B, crout)
    print("---LU Crouta took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    solve_equtition(A, B, dollitle)
    print("---LU Doolitla took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    solve_equtition(A, B, cholesky)
    print("---LU Choleskyego took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    np.linalg.solve(A, B)
    print("---Numpy solver took %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    scipy.linalg.solve(A, B)
    print("---Scipy solver took %s seconds ---" % (time.time() - start_time))

Wyniki testów wykonałem poza środowiskiem jupytera, a same wyniki są nastepujące :

- Dla macierzy 100x100 zakres losowanych liczb [0, 500] :    
  ---LU Crouta took 0.05288290977478027 seconds ---           
  ---LU Doolitla took 0.19524312019348145 seconds ---           
  ---LU Choleskyego took 0.1520702838897705 seconds ---         
  ---Numpy solver took 0.0009083747863769531 seconds ---        
  ---Scipy solver took 0.0009958744049072266 seconds ---        
- Dla macierzy 200x200 zakres losowanych liczb [0, 500] :         
  ---LU Crouta took 0.2916877269744873 seconds ---          
  ---LU Doolitla took 0.7662258148193359 seconds ---           
  ---LU Choleskyego took 0.41582250595092773 seconds ---       
  ---Numpy solver took 0.0016617774963378906 seconds ---          
  ---Scipy solver took 0.0023343563079833984 seconds ---       
- Dla macierzy 300x300 zakres losowanych liczb [0, 500] :              
  ---LU Crouta took 0.9223763942718506 seconds ---             
  ---LU Doolitla took 2.279259443283081 seconds ---           
  ---LU Choleskyego took 1.1543560028076172 seconds ---              
  ---Numpy solver took 0.0030765533447265625 seconds ---            
  ---Scipy solver took 0.004083871841430664 seconds ---           
- Dla macierzy 500x500 zakres losowanych liczb [0, 500] :         
  ---LU Crouta took 4.130779504776001 seconds ---           
  ---LU Doolitla took 10.669626712799072 seconds ---           
  ---LU Choleskyego took 4.949918985366821 seconds ---                    
  ---Numpy solver took 0.006085872650146484 seconds ---       
  ---Scipy solver took 0.009285449981689453 seconds ---           
- Dla macierzy 1000x1000 zakres losowanych liczb [0, 500] :       
  ---LU Crouta took 36.34651827812195 seconds ---        
  ---LU Doolitla took 86.19828629493713 seconds ---                   
  ---LU Choleskyego took 40.54396080970764 seconds ---             
  ---Numpy solver took 0.016066551208496094 seconds ---              
  ---Scipy solver took 0.02889227867126465 seconds ---                    
